## Content

This dataset consists of real-world consumer complaints about financial products and services. Each complaint is labeled with a specific product, framing this as a supervised text classification problem. To classify future complaints based on their content, various machine learning algorithms were employed to improve the accuracy of predictions, assigning each complaint to the appropriate product category.

In [1]:
# Importing Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import randint
from io import StringIO
import gc

# Preprocessing
from sklearn.model_selection import train_test_split

# Feature selection
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfTransformer

# Actual models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

# Evaluation metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

# Visualization
from IPython.display import display
import seaborn as sns # used for plot interactive graph.
import matplotlib.pyplot as plt
import seaborn as sns

## Data Preparation

In [ ]:
df = pd.read_csv("https://storage.googleapis.com/suptech-lab-practical-data-science-public/complaints.csv.zip")
df.shape